# Experimenting with Package Functions

IN PROGRESS

## (1) Data generation with MOSFIT

In [ ]:
from survey_agnostic_sn_vae.data_generation.mosfit2 import *
from survey_agnostic_sn_vae.data_generation.objects import *
import os

OUTPUT_PATH = '/Users/kdesoto/multisurvey_data'
lsst_survey = Survey(
    'LSST',
    ['u', 'g', 'r', 'i', 'z', 'y'],
    3
)
generate_LCs_from_model(
    'slsn',
    [lsst_survey,],
    100,
    output_path=OUTPUT_PATH
)

/Users/kdesoto/multisurvey_data
Switching to MOSFIT path: /Users/kdesoto/python_repos/MOSFiT/mosfit

Basic model file:
  models/model.json
Model file:
  models/slsn/slsn.json
Parameter file:
  models/slsn/parameters.json
No event specified, generating dummy data.

Loading data...

Loading module `transient` [ 0% ]

Loading module `lumdist` [ 2% ]

Loading module `redshift` [ 5% ]

Loading module `alltimes` [ 7% ]

Loading module `texplosion` [ 10% ]

Loading module `resttimes` [ 12% ]

Loading module `densetimes` [ 15% ]

Loading module `Pspin` [ 17% ]

Loading module `Bfield` [ 20% ]

Loading module `Mns` [ 22% ]

Loading module `thetaPB` [ 24% ]

Loading module `mejecta` [ 27% ]

Loading module `kappa` [ 29% ]

Loading module `kappagamma` [ 32% ]

Loading module `vejecta` [ 34% ]

Loading module `magnetar` [ 37% ]

Loading module `temperature` [ 39% ]

Loading module `diffusion` [ 41% ]

Loading module `cutoff_wavelength` [ 44% ]

Loading module `temperature_floor` [ 46% ]

Loading m